# Basic text processing

The purpose of this notebook is to introduce you to some basic concepts and techniques for processing text data.

## Dataset

The dataset for this notebook is [WikiText](https://www.salesforce.com/products/einstein/ai-research/the-wikitext-dependency-language-modeling-dataset/), which is based on text extracted from “Good” and “Featured” articles on English Wikipedia. More specifically, we will be using the training portion of the WikiText-2 dataset.

We start by opening the file and store its contents in a list:

In [ ]:
with open("wiki.train.tokens", encoding="utf-8") as f:
    wikitext = f.readlines()

Here is a sample line from the file:

In [ ]:
wikitext[1234]

#### 🎈 Task 1: A close look at the data

Take a close look at this line and compare it with the [original](https://en.wikipedia.org/wiki/The_Feast_of_the_Goat#Critical_reception). You will notice several differences between the two texts. What are your hypotheses as to how these differences come about?

## Tokenisation

Many tasks in natural language processing rely on first segmenting the raw text into words or word-like units. This task is called **tokenisation**.

In real applications, we would have to tokenise the raw text ourselves, perhaps using some library such as [spaCy](https://spacy.io) or [Stanford CoreNLP](https://stanfordnlp.github.io/CoreNLP/). The WikiText dataset, however, is pre-tokenised, so that we can extract the tokens from a given line using a simple helper function that splits on whitespace:

In [ ]:
def tokens(lines):
    for line in lines:
        for token in line.split():
            yield token

With this, we can count the number of tokens in the dataset:

In [ ]:
sum(1 for t in tokens(wikitext))

## Vocabulary

The term *word* is used ambiguously in natural language processing. It can either refer to a *token*, a word as it occurs in the running text, or to a *unique word*. We are interested in tokens when we want to count the length of a text, and in unique words when we want to compile a dictionary of all words in the text material.

The set of unique words in a dataset or task is called the **vocabulary**.

In [ ]:
vocab = set(tokens(wikitext))

Show the size of the vocabulary:

In [ ]:
len(vocab)

#### 🎈 Task 2: Missing words

When you compare this number to the corresponding number on the [WikiText website](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/), you will notice a small difference. Try to find out what causes this difference, and how you can change the code so that you get the same number as on the website.

For some natural language processing tasks, tokenisation is the only thing we need to do in order to compile the vocabulary. For other tasks, we may want to apply additional pre-processing or apply some frequency cut-off.

#### 🎈 Task 3: Normalisation

The term **text normalisation** refers to all kinds of processing that we apply to a text in order to bring it into a more standard, convenient form. Two standard normalisation techniques are lowercasing and the filtering-out of non-alphabetical tokens. What effect do these techniques have on the size of the vocabulary?

## String-to-index mapping

One common task when processing text with neural networks is to encode words and other strings into integer IDs, so that we can map them to the components of a vector. A standard pattern for doing this looks as follows:

In [ ]:
w2i = {}
for token in tokens(wikitext):
    if token not in w2i:
        w2i[token] = len(w2i)

Assuming that we already have the vocabulary, we can also use a different pattern:

In [ ]:
w2i = {w: i for i, w in enumerate(sorted(vocab))}

What index did the word *language* get?

In [ ]:
w2i["language"]

Sometimes we also need to invert the string-to-integer mapping:

In [ ]:
i2w = {i: s for s, i in w2i.items()}

Decoding should be the inverse of encoding:

In [ ]:
assert i2w[w2i["language"]] == "language"

## Power-law behaviour of language

Language data often exhibits a power-law behaviour. We want to check whether this also applies to the WikiText data. In particular, we want to check whether the word frequencies follow the expected Zipfian pattern. We start by collecting the frequencies:

In [ ]:
from collections import Counter

counter = Counter(tokens(wikitext))

How often does the word *language* occur in the data?

In [ ]:
counter["language"]

Prepare the plotting:

In [ ]:
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = "svg"

Produce the rank/frequency plot for the 30,000 most common words:

In [ ]:
words, counts = zip(*counter.most_common(30000))

plt.xlabel("Rank")
plt.yscale("log")
plt.ylabel("Frequency")
plt.plot(range(len(words)), counts)
plt.show()

#### 🎈 Task 4: Visualise Heaps’ law

Produce a plot showing the number of words encountered after reading a certain number of tokens in the WikiText dataset. How many new words, approximately, appear in the second half of the text?

That’s all, folks!